# Day 8_0: 시계열 기초 (Time Series Fundamentals)

## 📚 학습 목표

**Part 1: 기초**
1. 시계열 데이터의 개념과 특성 이해
2. `pd.to_datetime()`으로 날짜 문자열 변환
3. `DatetimeIndex` 설정 및 활용
4. 시간대(timezone) 처리 방법
5. 날짜 범위 생성 (`date_range`)

**Part 2: 심화**
1. 시계열 인덱싱 및 슬라이싱 마스터
2. `shift()`로 시차 데이터 생성
3. `diff()`로 변화량 계산
4. `px.line()`으로 시계열 시각화

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 시계열 데이터 | 시간에 따른 패턴 분석 | 주가 변동, 매출 추이, 웹 트래픽 |
| datetime 변환 | 날짜 문자열을 분석 가능한 형태로 | "2024-01-15" → Timestamp 객체 |
| DatetimeIndex | 날짜 기반 필터링/집계 | 특정 월/연도 데이터 추출 |
| shift/diff | 전일 대비 변화율 계산 | 일별 매출 증가율, 주가 등락폭 |
| 시계열 시각화 | 트렌드 및 이상치 발견 | Plotly로 인터랙티브 차트 |

**분석가 관점**: 시계열 데이터는 비즈니스의 과거-현재-미래를 연결하는 핵심 자산으로, 올바른 처리가 정확한 예측의 첫걸음입니다.

---

## Part 1: 기초

### 1.1 시계열 데이터란?

시계열 데이터는 **시간 순서대로 기록된 데이터**입니다. 핵심 특징:
- **시간 의존성**: 과거 값이 미래에 영향
- **순서 중요**: 데이터 순서 바뀌면 의미 손실
- **주기성/계절성**: 반복 패턴 존재 가능

**일반 데이터 vs 시계열 데이터**:

| 구분 | 일반 데이터 | 시계열 데이터 |
|------|-----------|-------------|
| 순서 | 독립적 (셔플 가능) | 시간 순서 필수 |
| 상관관계 | 행 간 독립 | 연속된 값 상관 |
| 분석 목표 | 분류/군집 | 추세/예측 |

💡 **실무 예시**: 전자상거래에서 매일의 주문량, 웹사이트 일별 방문자 수, 주식의 일봉/시봉 데이터 모두 시계열입니다.

In [16]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

# 시계열 데이터 예제: 일주일간 매출
sales_data = {
    'date': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05'],
    'revenue': [1200, 1350, 1180, 1420, 1590]  # 단위: 만원
}
df = pd.DataFrame(sales_data)
print("📊 원본 데이터:")
print(df)
print(f"\ndate 컬럼 타입: {df['date'].dtype}")  # object (문자열)

📊 원본 데이터:
         date  revenue
0  2024-01-01     1200
1  2024-01-02     1350
2  2024-01-03     1180
3  2024-01-04     1420
4  2024-01-05     1590

date 컬럼 타입: object


---

### 1.2 pd.to_datetime() - 날짜 변환의 시작

문자열로 된 날짜를 pandas `Timestamp` 객체로 변환합니다.        #df.to_datetime할 것 같지만 이건 instance method고 지금 이건 class method단계임.


**주요 파라미터**:
- `format`: 날짜 형식 지정 (`'%Y-%m-%d'`, `'%Y/%m/%d'` 등)
- `errors`: 변환 실패 시 처리 (`'raise'`, `'coerce'`, `'ignore'`)
- `dayfirst`: 일/월 순서 (유럽식 날짜)

💡 **실무 예시**: 웹 스크레이핑으로 수집한 뉴스 날짜 "2024.01.15" → 분석 가능한 datetime 객체

In [17]:
# 1) 기본 변환
df['date'] = pd.to_datetime(df['date'])
print("✅ 변환 후:")
print(df)
print(f"\ndate 컬럼 타입: {df['date'].dtype}")  # datetime64[ns]

# 2) 다양한 형식 처리
date_formats = {
    'slash': '2024/01/15',
    'dot': '2024.01.15',
    'korean': '2024년 1월 15일',
    'timestamp': '2024-01-15 14:30:25'
}

for name, date_str in date_formats.items():
    try:
        converted = pd.to_datetime(date_str)
        print(f"{name:10s}: {date_str:25s} → {converted}")
    except:
        print(f"{name:10s}: {date_str:25s} → ❌ 변환 실패")

✅ 변환 후:
        date  revenue
0 2024-01-01     1200
1 2024-01-02     1350
2 2024-01-03     1180
3 2024-01-04     1420
4 2024-01-05     1590

date 컬럼 타입: datetime64[ns]
slash     : 2024/01/15                → 2024-01-15 00:00:00
dot       : 2024.01.15                → 2024-01-15 00:00:00
korean    : 2024년 1월 15일              → ❌ 변환 실패
timestamp : 2024-01-15 14:30:25       → 2024-01-15 14:30:25


In [18]:
# 3) 에러 처리 옵션
messy_dates = ['2024-01-15', 'invalid', '2024-02-20', 'wrong']

# errors='coerce': 변환 실패 시 NaT (Not a Time)
converted_dates = pd.to_datetime(messy_dates, errors='coerce') #변환실패시 NaT로 변환
print("errors='coerce' 결과:")
print(converted_dates)
print(f"\n❌ 변환 실패 개수: {converted_dates.isna().sum()}개")

errors='coerce' 결과:
DatetimeIndex(['2024-01-15', 'NaT', '2024-02-20', 'NaT'], dtype='datetime64[ns]', freq=None)

❌ 변환 실패 개수: 2개


---

### 1.3 DatetimeIndex 설정

`DatetimeIndex`는 날짜를 DataFrame의 인덱스로 사용하여:
- 날짜 기반 슬라이싱 가능 (`df['2024-01']`)
- 시계열 전용 메서드 활성화 (`resample`, `shift` 등)
- 자동 정렬 및 병합 지원

💡 **실무 예시**: 주식 데이터에서 특정 연월 수익률 계산 시 `df.loc['2024-01']`로 간편 필터링

In [19]:
# DatetimeIndex 설정
df_indexed = df.set_index('date')
print("📅 DatetimeIndex 적용:")
print(df_indexed)
print(f"\n인덱스 타입: {type(df_indexed.index)}")

# 인덱스 속성 확인
print("\n📊 인덱스 정보:")
print(f"시작일: {df_indexed.index.min()}")
print(f"종료일: {df_indexed.index.max()}")
print(f"데이터 기간: {(df_indexed.index.max() - df_indexed.index.min()).days}일")

📅 DatetimeIndex 적용:
            revenue
date               
2024-01-01     1200
2024-01-02     1350
2024-01-03     1180
2024-01-04     1420
2024-01-05     1590

인덱스 타입: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>

📊 인덱스 정보:
시작일: 2024-01-01 00:00:00
종료일: 2024-01-05 00:00:00
데이터 기간: 4일


---

### 1.4 시간대 (Timezone) 처리

글로벌 서비스에서 중요한 시간대 관리:
- `tz_localize()`: 시간대 정보 추가 (naive → aware)
- `tz_convert()`: 다른 시간대로 변환

💡 **실무 예시**: AWS 서버 로그(UTC)를 한국 시간(Asia/Seoul)으로 변환하여 분석

In [20]:
# 시간대 없는 시간 (naive)
naive_time = pd.Timestamp('2024-01-15 14:30:00')
print(f"Naive 시간: {naive_time}")
print(f"시간대 정보: {naive_time.tz}\n")  # None

# 시간대 지정 (aware)
seoul_time = naive_time.tz_localize('Asia/Seoul')
print(f"Seoul 시간: {seoul_time}")
print(f"시간대 정보: {seoul_time.tz}\n")

# 시간대 변환
utc_time = seoul_time.tz_convert('UTC')
ny_time = seoul_time.tz_convert('America/New_York')

print("🌍 동일 시점의 세계 시간:")
print(f"서울: {seoul_time}")
print(f"UTC:  {utc_time}")
print(f"뉴욕: {ny_time}")

Naive 시간: 2024-01-15 14:30:00
시간대 정보: None

Seoul 시간: 2024-01-15 14:30:00+09:00
시간대 정보: Asia/Seoul

🌍 동일 시점의 세계 시간:
서울: 2024-01-15 14:30:00+09:00
UTC:  2024-01-15 05:30:00+00:00
뉴욕: 2024-01-15 00:30:00-05:00


---

### 1.5 날짜 범위 생성 (date_range)

`pd.date_range()`로 연속된 날짜 생성:
- `start`, `end`: 시작/종료일
- `periods`: 생성할 날짜 개수
- `freq`: 빈도 (`'D'`=일, `'M'`=월말, `'H'`=시간 등)

💡 **실무 예시**: 월별 예산 계획 테이블 생성 시 12개월 날짜 자동 생성

In [21]:
# 1) 일별 날짜 범위 (30일)
daily_dates = pd.date_range(start='2024-01-01', periods=30, freq='D')
print("📅 일별 30일:")
print(daily_dates[:5])  # 처음 5개만 표시
print(f"총 {len(daily_dates)}개\n")

# 2) 영업일만 (주말 제외)
business_dates = pd.date_range(start='2024-01-01', periods=20, freq='B')
print("📊 영업일 20일:")
print(business_dates[:5])
print(f"총 {len(business_dates)}개\n")

# 3) 월말 날짜
month_end_dates = pd.date_range(start='2024-01-31', periods=12, freq='ME')
print("📆 월말 12개월:")
print(month_end_dates)

# 4) 시간 단위
hourly_dates = pd.date_range(start='2024-01-15 09:00', periods=8, freq='H')
print("\n⏰ 시간별 8시간:")
print(hourly_dates)

📅 일별 30일:
DatetimeIndex(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04',
               '2024-01-05'],
              dtype='datetime64[ns]', freq='D')
총 30개

📊 영업일 20일:
DatetimeIndex(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04',
               '2024-01-05'],
              dtype='datetime64[ns]', freq='B')
총 20개

📆 월말 12개월:
DatetimeIndex(['2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30', '2024-07-31', '2024-08-31',
               '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31'],
              dtype='datetime64[ns]', freq='ME')

⏰ 시간별 8시간:
DatetimeIndex(['2024-01-15 09:00:00', '2024-01-15 10:00:00',
               '2024-01-15 11:00:00', '2024-01-15 12:00:00',
               '2024-01-15 13:00:00', '2024-01-15 14:00:00',
               '2024-01-15 15:00:00', '2024-01-15 16:00:00'],
              dtype='datetime64[ns]', freq='h')


/tmp/ipykernel_23669/2664235778.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_dates = pd.date_range(start='2024-01-15 09:00', periods=8, freq='H')


In [22]:
# 실무 활용: 월별 매출 목표 데이터 생성
monthly_index = pd.date_range(start='2024-01', periods=12, freq='MS')  # Month Start
monthly_target = pd.DataFrame({
    'target_revenue': np.random.randint(5000, 8000, 12)  # 5000-8000만원
}, index=monthly_index)

print("📊 2024년 월별 매출 목표:")
print(monthly_target)

📊 2024년 월별 매출 목표:
            target_revenue
2024-01-01            5255
2024-02-01            5708
2024-03-01            6838
2024-04-01            6944
2024-05-01            7497
2024-06-01            5009
2024-07-01            7871
2024-08-01            5797
2024-09-01            5876
2024-10-01            6028
2024-11-01            6015
2024-12-01            6824


---

## Part 2: 시계열 조작

### 2.1 시계열 인덱싱 및 슬라이싱

DatetimeIndex의 강력한 기능:
- 부분 문자열 매칭: `df['2024']`, `df['2024-01']`
- 범위 슬라이싱: `df['2024-01':'2024-03']`
- 조건 필터링: `df[df.index.month == 1]`

💡 **실무 예시**: 분기별 실적 보고 시 `df['2024-Q1']` 형태로 3개월 데이터 한번에 추출

In [23]:
# 2024년 전체 일별 매출 데이터 생성
np.random.seed(42)
date_range_2024 = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')
daily_revenue = pd.DataFrame({
    'revenue': np.random.randint(1000, 3000, len(date_range_2024)),  # 1000-3000만원
    'orders': np.random.randint(50, 200, len(date_range_2024))
}, index=date_range_2024)

print("📊 2024년 일별 매출 데이터 (샘플):")
print(daily_revenue.head(10))
print(f"\n총 {len(daily_revenue)}일 데이터")

📊 2024년 일별 매출 데이터 (샘플):
            revenue  orders
2024-01-01     2126     109
2024-01-02     2459     162
2024-01-03     1860      51
2024-01-04     2294     178
2024-01-05     2130      97
2024-01-06     2095     189
2024-01-07     2724      86
2024-01-08     2044      58
2024-01-09     2638     148
2024-01-10     1121     196

총 366일 데이터


In [ ]:
# 1) 특정 월 데이터 추출
jan_data = daily_revenue['2024-01':]    #==> ⭐이거 원래 '2024-01'이었는데 오류나서 고침.⭐ 더 알아볼것!!
print("📅 2024년 1월 데이터:")
print(jan_data.head())
print(f"1월 총 매출: {jan_data['revenue'].sum():,}만원\n")

# 2) 범위 슬라이싱 (1분기)
q1_data = daily_revenue['2024-01':'2024-03']
print("📊 2024년 1분기 (1-3월):")
print(f"데이터 기간: {q1_data.index.min()} ~ {q1_data.index.max()}")
print(f"1분기 총 매출: {q1_data['revenue'].sum():,}만원\n")

# 3) 특정 날짜
specific_day = daily_revenue.loc['2024-06-15']
print("📆 2024년 6월 15일 실적:")
print(specific_day)

📅 2024년 1월 데이터:
            revenue  orders
2024-01-01     2126     109
2024-01-02     2459     162
2024-01-03     1860      51
2024-01-04     2294     178
2024-01-05     2130      97
1월 총 매출: 752,429만원

📊 2024년 1분기 (1-3월):
데이터 기간: 2024-01-01 00:00:00 ~ 2024-03-31 00:00:00
1분기 총 매출: 179,500만원

📆 2024년 6월 15일 실적:
revenue    2597
orders      138
Name: 2024-06-15 00:00:00, dtype: int64


In [ ]:
# 4) 조건 필터링: 월별/요일별
# 여름 시즌 (6-8월) 매출
summer_sales = daily_revenue[daily_revenue.index.month.isin([6, 7, 8])] #6,7,8월에 속하는지 ? 
print("☀️ 여름 시즌 (6-8월) 매출:")
print(f"평균 일매출: {summer_sales['revenue'].mean():.0f}만원\n")

# 주말 매출 (토요일=5, 일요일=6)
weekend_sales = daily_revenue[daily_revenue.index.dayofweek.isin([5, 6])]       # 0 1 2 3 4 5 6 순서라서 토,일이 5,6임. 
print("🎉 주말 매출:")
print(f"주말 평균 매출: {weekend_sales['revenue'].mean():.0f}만원")
print(f"평일 평균 매출: {daily_revenue[~daily_revenue.index.dayofweek.isin([5, 6])]['revenue'].mean():.0f}만원")

---

### ⭐⭐⭐2.2 shift() - 시차 데이터 생성⭐⭐⭐

`shift()`는 데이터를 시간축으로 이동:
- `shift(1)`: 1기간 뒤로 이동 (전일 값)
- `shift(-1)`: 1기간 앞으로 이동 (다음날 값)
- 전일 대비, 전월 대비 계산에 필수

💡 **실무 예시**: 주가 데이터에서 `today_price - shift(1)` → 일일 등락폭 계산

In [ ]:
# 주식 종가 예제
stock_dates = pd.date_range(start='2024-01-02', periods=10, freq='B')
stock_df = pd.DataFrame({
    'close_price': [52000, 52500, 51800, 53200, 53800, 54200, 53900, 54500, 55100, 54800]
}, index=stock_dates)

print("📈 주식 종가 데이터:")
print(stock_df)


📈 주식 종가 데이터:
            close_price
2024-01-02        52000
2024-01-03        52500
2024-01-04        51800
2024-01-05        53200
2024-01-08        53800
2024-01-09        54200
2024-01-10        53900
2024-01-11        54500
2024-01-12        55100
2024-01-15        54800


In [ ]:

# shift() 적용
stock_df['prev_close'] = stock_df['close_price'].shift(1)  # 전일 종가
stock_df['next_close'] = stock_df['close_price'].shift(-1)  # 다음날 종가

print("\n🔄 shift() 결과:")
print(stock_df)


🔄 shift() 결과:
            close_price  prev_close  next_close
2024-01-02        52000         NaN     52500.0
2024-01-03        52500     52000.0     51800.0
2024-01-04        51800     52500.0     53200.0
2024-01-05        53200     51800.0     53800.0
2024-01-08        53800     53200.0     54200.0
2024-01-09        54200     53800.0     53900.0
2024-01-10        53900     54200.0     54500.0
2024-01-11        54500     53900.0     55100.0
2024-01-12        55100     54500.0     54800.0
2024-01-15        54800     55100.0         NaN


In [ ]:
# 전일 대비 등락률 계산
stock_df['change_rate'] = ((stock_df['close_price'] - stock_df['prev_close']) / stock_df['prev_close'] * 100).round(2)

print("📊 전일 대비 등락률:")
print(stock_df[['close_price', 'prev_close', 'change_rate']])

# 통계
print(f"\n평균 등락률: {stock_df['change_rate'].mean():.2f}%")
print(f"최대 상승일: {stock_df['change_rate'].max():.2f}%")
print(f"최대 하락일: {stock_df['change_rate'].min():.2f}%")

📊 전일 대비 등락률:
            close_price  prev_close  change_rate
2024-01-02        52000         NaN          NaN
2024-01-03        52500     52000.0         0.96
2024-01-04        51800     52500.0        -1.33
2024-01-05        53200     51800.0         2.70
2024-01-08        53800     53200.0         1.13
2024-01-09        54200     53800.0         0.74
2024-01-10        53900     54200.0        -0.55
2024-01-11        54500     53900.0         1.11
2024-01-12        55100     54500.0         1.10
2024-01-15        54800     55100.0        -0.54

평균 등락률: 0.59%
최대 상승일: 2.70%
최대 하락일: -1.33%


---

### 2.3 diff() - 변화량 계산

`diff()`는 연속된 값의 차이 계산:
- `diff(1)`: 이전 값과의 차이 (= `value - shift(1)`)        #이런걸 차분이라고 함! 
- `diff(n)`: n기간 전 값과의 차이

⚠️ **shift vs diff**:
- `shift()`: 값 자체 이동 → 원본 값 보존
- `diff()`: 차이 계산 → 변화량만 표시

💡 **실무 예시**: 일일 신규 가입자 수(누적 회원 수의 `diff()`)를 구하여 마케팅 효과 분석

In [ ]:
# 누적 회원 수 예제
member_dates = pd.date_range(start='2024-01-01', periods=10, freq='D')
member_df = pd.DataFrame({
    'total_members': [1000, 1025, 1053, 1078, 1095, 1120, 1158, 1180, 1205, 1230]
}, index=member_dates)

print("👥 누적 회원 수:")
print(member_df)

👥 누적 회원 수:
            total_members
2024-01-01           1000
2024-01-02           1025
2024-01-03           1053
2024-01-04           1078
2024-01-05           1095
2024-01-06           1120
2024-01-07           1158
2024-01-08           1180
2024-01-09           1205
2024-01-10           1230


In [ ]:

# diff()로 일일 신규 가입자 계산
member_df['new_members'] = member_df['total_members'].diff()

print("\n✨ 일일 신규 가입자:")
print(member_df)


✨ 일일 신규 가입자:
            total_members  new_members
2024-01-01           1000          NaN
2024-01-02           1025         25.0
2024-01-03           1053         28.0
2024-01-04           1078         25.0
2024-01-05           1095         17.0
2024-01-06           1120         25.0
2024-01-07           1158         38.0
2024-01-08           1180         22.0
2024-01-09           1205         25.0
2024-01-10           1230         25.0


In [ ]:
# shift vs diff 비교
comparison_df = pd.DataFrame({
    'value': [100, 105, 102, 108, 110]
}, index=pd.date_range('2024-01-01', periods=5, freq='D'))

comparison_df['shift_1'] = comparison_df['value'].shift(1)
comparison_df['diff_1'] = comparison_df['value'].diff(1)
comparison_df['manual_diff'] = comparison_df['value'] - comparison_df['shift_1']

print("🔍 shift vs diff:")
print(comparison_df)
print("\n✅ diff(1) == value - shift(1)")

---

### 2.4 px.line() - 시계열 시각화

Plotly Express의 `px.line()`으로 인터랙티브 시계열 차트:
- 줌, 패닝, 호버 정보 자동 지원
- 여러 시리즈 동시 표시 가능
- 마커, 선 스타일 커스터마이징

💡 **실무 예시**: 경영진 보고용 대시보드에서 월별 매출 추이를 인터랙티브 차트로 제공

In [26]:
pd.options.plotting.backend = "plotly"

In [27]:
# 월별 매출 데이터 생성
monthly_dates = pd.date_range(start='2024-01', periods=12, freq='MS')
monthly_revenue = pd.DataFrame({
    'revenue': [5200, 5800, 6100, 6500, 7200, 7800, 8100, 7900, 7500, 7200, 8500, 9200],
    'cost': [3100, 3400, 3600, 3800, 4200, 4600, 4800, 4700, 4400, 4200, 5000, 5400]
}, index=monthly_dates)

monthly_revenue['profit'] = monthly_revenue['revenue'] - monthly_revenue['cost']

print("📊 2024년 월별 재무 데이터:")
print(monthly_revenue)

📊 2024년 월별 재무 데이터:
            revenue  cost  profit
2024-01-01     5200  3100    2100
2024-02-01     5800  3400    2400
2024-03-01     6100  3600    2500
2024-04-01     6500  3800    2700
2024-05-01     7200  4200    3000
2024-06-01     7800  4600    3200
2024-07-01     8100  4800    3300
2024-08-01     7900  4700    3200
2024-09-01     7500  4400    3100
2024-10-01     7200  4200    3000
2024-11-01     8500  5000    3500
2024-12-01     9200  5400    3800


In [28]:
# 1) 기본 선 그래프
fig1 = px.line(
    monthly_revenue.reset_index(), 
    x='index', 
    y='revenue',
    title='📈 2024년 월별 매출 추이',
    labels={'index': '월', 'revenue': '매출 (만원)'},
    markers=True  # 데이터 포인트 표시
)
fig1.update_traces(line_color='#1f77b4', line_width=3)
fig1.show()

In [29]:
# 2) 여러 시리즈 동시 표시
fig2 = go.Figure()

# 매출
fig2.add_trace(go.Scatter(
    x=monthly_revenue.index,
    y=monthly_revenue['revenue'],
    mode='lines+markers',
    name='매출',
    line=dict(color='#2ecc71', width=3)
))

# 비용
fig2.add_trace(go.Scatter(
    x=monthly_revenue.index,
    y=monthly_revenue['cost'],
    mode='lines+markers',
    name='비용',
    line=dict(color='#e74c3c', width=3, dash='dash')
))

# 순이익
fig2.add_trace(go.Scatter(
    x=monthly_revenue.index,
    y=monthly_revenue['profit'],
    mode='lines+markers',
    name='순이익',
    line=dict(color='#f39c12', width=3)
))

fig2.update_layout(
    title='📊 2024년 재무 현황',
    xaxis_title='월',
    yaxis_title='금액 (만원)',
    hovermode='x unified',
    height=500
)
fig2.show()

In [31]:
# 3) 실무 활용: 일별 매출 + 이동평균
# 1월 데이터만 사용
jan_sales = daily_revenue['2024-01' : '2024-01'].copy()
jan_sales['ma_7'] = jan_sales['revenue'].rolling(window=7).mean()  # 7일 이동평균

fig3 = go.Figure()

# 일별 매출 (투명도 낮춤)
fig3.add_trace(go.Scatter(
    x=jan_sales.index,
    y=jan_sales['revenue'],
    mode='lines',
    name='일별 매출',
    line=dict(color='lightgray', width=1),
    opacity=0.5
))

# 7일 이동평균 (강조)
fig3.add_trace(go.Scatter(
    x=jan_sales.index,
    y=jan_sales['ma_7'],
    mode='lines',
    name='7일 이동평균',
    line=dict(color='#3498db', width=4)
))

fig3.update_layout(
    title='📈 1월 일별 매출 및 추세 (7일 이동평균)',
    xaxis_title='날짜',
    yaxis_title='매출 (만원)',
    hovermode='x unified',
    height=500
)
fig3.show()

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 날짜 문자열 변환 ⭐

**문제**: 다음 날짜 문자열 리스트를 datetime 객체로 변환하세요. 변환 실패 시 NaT로 처리하세요.

```python
date_strings = ['2024-01-15', '2024/02/20', 'invalid_date', '2024-03-25']
```

**기대 결과**: DatetimeIndex 또는 Series, NaT 1개 포함

In [ ]:
# 여기에 코드를 작성하세요


---

### Q2. DatetimeIndex 설정 ⭐⭐

**문제**: 아래 DataFrame의 `date` 컬럼을 DatetimeIndex로 설정하고, 인덱스의 최소/최대 날짜를 출력하세요.

```python
data = {
    'date': ['2024-01-01', '2024-01-05', '2024-01-10', '2024-01-15'],
    'sales': [1200, 1500, 1350, 1600]
}
df_q2 = pd.DataFrame(data)
```

**기대 결과**: 
```
최소 날짜: 2024-01-01 00:00:00
최대 날짜: 2024-01-15 00:00:00
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q3. 영업일 날짜 범위 생성 ⭐⭐

**문제**: 2024년 1월 1일부터 20개의 영업일(주말 제외) 날짜를 생성하고, 첫 5개와 마지막 5개 날짜를 출력하세요.

**기대 결과**: DatetimeIndex 20개 (토/일 제외)

In [ ]:
# 여기에 코드를 작성하세요


---

### Q4. 시간대 변환 ⭐⭐

**문제**: 서울 시간 `2024-01-15 14:00:00`을 뉴욕 시간(America/New_York)으로 변환하세요.

**기대 결과**: 2024-01-15 00:00:00-05:00 (대략)

In [ ]:
# 여기에 코드를 작성하세요


---

### Q5. 특정 월 데이터 필터링 ⭐⭐⭐

**문제**: 아래 DataFrame에서 2024년 3월 데이터만 추출하고, 해당 월의 평균 매출을 계산하세요.

```python
dates = pd.date_range(start='2024-01-01', end='2024-06-30', freq='D')
df_q5 = pd.DataFrame({
    'revenue': np.random.randint(1000, 2000, len(dates))
}, index=dates)
```

**기대 결과**: 
```
3월 데이터: 31개
3월 평균 매출: 1500만원 (대략)
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q6. shift()로 전일 매출 비교 ⭐⭐⭐

**문제**: 아래 DataFrame에 전일 매출(`prev_revenue`)과 전일 대비 증가액(`change`) 컬럼을 추가하세요.

```python
dates = pd.date_range(start='2024-01-01', periods=7, freq='D')
df_q6 = pd.DataFrame({
    'revenue': [1200, 1350, 1180, 1420, 1590, 1500, 1650]
}, index=dates)
```

**기대 결과**: 2개의 새 컬럼 (`prev_revenue`, `change`)

In [ ]:
# 여기에 코드를 작성하세요


---

### Q7. diff()로 일일 신규 가입자 계산 ⭐⭐⭐

**문제**: 누적 회원 수에서 일일 신규 가입자를 계산하고, 가장 많은 신규 가입자가 발생한 날짜와 수를 출력하세요.

```python
dates = pd.date_range(start='2024-01-01', periods=10, freq='D')
df_q7 = pd.DataFrame({
    'total_members': [5000, 5025, 5060, 5095, 5110, 5150, 5185, 5200, 5240, 5275]
}, index=dates)
```

**기대 결과**: 
```
최대 신규 가입일: 2024-01-06
신규 가입자 수: 40명
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q8. 주말/평일 매출 비교 ⭐⭐⭐⭐

**문제**: 2024년 1월 전체 데이터에서 주말(토/일) 평균 매출과 평일 평균 매출을 계산하고, 차이를 출력하세요.

```python
np.random.seed(42)
dates = pd.date_range(start='2024-01-01', end='2024-01-31', freq='D')
df_q8 = pd.DataFrame({
    'revenue': np.random.randint(1000, 2500, len(dates))
}, index=dates)
```

**기대 결과**: 
```
평일 평균: 1650만원
주말 평균: 1820만원
차이: 170만원
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q9. px.line() 멀티 시리즈 시각화 ⭐⭐⭐⭐

**문제**: 아래 DataFrame의 `매출`, `비용`, `순이익`을 하나의 그래프에 선으로 표시하세요. 범례 포함.

```python
months = pd.date_range(start='2024-01', periods=6, freq='MS')
df_q9 = pd.DataFrame({
    '매출': [5000, 5500, 6000, 6200, 6800, 7200],
    '비용': [3000, 3200, 3500, 3600, 4000, 4200],
    '순이익': [2000, 2300, 2500, 2600, 2800, 3000]
}, index=months)
```

**기대 결과**: 3개의 선이 표시된 인터랙티브 차트

In [ ]:
# 여기에 코드를 작성하세요


---

### Q10. 종합 분석: 매출 전월 대비 증가율 ⭐⭐⭐⭐⭐

**문제**: 2024년 월별 매출 데이터에서:
1. 전월 대비 증가율(%) 계산
2. 증가율이 가장 높은 달 찾기
3. 매출과 증가율을 하나의 차트에 시각화 (2축 사용)

```python
months = pd.date_range(start='2024-01', periods=12, freq='MS')
df_q10 = pd.DataFrame({
    'revenue': [5000, 5200, 5800, 6100, 6500, 7200, 7800, 7600, 7400, 7800, 8500, 9200]
}, index=months)
```

**기대 결과**: 
```
최고 증가율 월: 2024-07
증가율: 8.33%
```
+ 2축 차트 (매출-막대, 증가율-선)

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메소드/문법 | 실무 활용 |
|------|-----------------|----------|
| 시계열 데이터 | 시간 순서, 의존성 | 주가, 매출, 트래픽 분석 |
| datetime 변환 | `pd.to_datetime()` | 문자열 날짜 → 분석 가능 객체 |
| DatetimeIndex | `df.set_index()` | 날짜 기반 필터링/슬라이싱 |
| 시간대 처리 | `tz_localize()`, `tz_convert()` | 글로벌 서비스 로그 분석 |
| 날짜 범위 생성 | `pd.date_range()` | 월별/일별 템플릿 생성 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| 시계열 인덱싱 | `df['2024-01']`, `df['2024-01':'2024-03']` | 특정 기간 데이터 추출 |
| shift() | `df['col'].shift(1)` | 전일/전월 값 비교 |
| diff() | `df['col'].diff()` | 변화량, 증가폭 계산 |
| px.line() | `px.line(df, x, y, markers=True)` | 시계열 추세 시각화 |

### 💡 실무 팁

1. **날짜 파싱 오류 대비**: `errors='coerce'`로 잘못된 날짜 NaT 처리 후 확인
2. **시간대 일관성**: 모든 데이터를 UTC로 통일 후 필요시 로컬 시간으로 변환
3. **영업일 계산**: `freq='B'`로 주말 자동 제외 (공휴일은 별도 처리 필요)
4. **shift vs diff 선택**: 원본 값 필요 시 `shift()`, 변화량만 필요 시 `diff()`
5. **시각화 성능**: 데이터가 많으면 월별 집계 후 시각화로 속도 개선
6. **인덱스 리셋**: Plotly에 넘길 때 `reset_index()`로 날짜를 컬럼으로 변환
7. **이동평균 활용**: 노이즈 많은 일별 데이터는 7일/30일 이동평균으로 추세 파악

---

**다음 학습**: Day 8_1에서 `resample()` (리샘플링), `rolling()` (이동평균), 시계열 분해를 배웁니다! 🚀